<a href="https://colab.research.google.com/github/aaronjoel/DeepUnderstandingOfDeepLearning/blob/main/DL_Addition_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

In [ ]:
## Generate the data needed for the change

# Generate 10000 random integers between -5000 and 5000
A = np.random.randint(low=-10000, high=10000, size=10000)
B = np.random.randint(low=-10000, high=10000, size=10000)
C = A + B

In [ ]:
data = pd.DataFrame({'A': A, 'B': B, 'C': C})
data.head()

,A,B,C
0,-2630,-9744,-12374
1,-7303,-7213,-14516
2,-1600,-5842,-7442
3,4676,4709,9385
4,-7938,-4447,-12385


In [ ]:
## Create the target & features vectors
X = data.drop('C', axis=1).values
y = data[['C']].values

In [ ]:
X = torch.tensor(X, dtype=torch.float)
y = torch.tensor(y, dtype=torch.float)

In [ ]:
# Split the data into train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([8000, 2]),
 torch.Size([2000, 2]),
 torch.Size([8000, 1]),
 torch.Size([2000, 1]))

In [ ]:
## Create the model
model = nn.Sequential(
    nn.Linear(2, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
)

loss_fun = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

In [ ]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, drop_last=True)
test_loader = DataLoader(test_data, batch_size=test_data.tensors[0].shape[0])

In [ ]:
train_loader.dataset.tensors[0].shape

torch.Size([8000, 2])

In [ ]:
epochs = 200

model.train()
for epoch in range(epochs):
  batch_losses = []
  for batch in train_loader:
    X_batch, y_batch = batch
    preds = model(X_batch)

    loss = loss_fun(preds, y_batch)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    batch_losses.append(loss.item())
  if epoch % 10 == 0:
    print(f'Epoch: {epoch}, Loss: {np.mean(batch_losses)}')


model.eval()
with torch.no_grad():
  for batch in test_loader:
    X_batch, y_batch = batch
    preds = model(X_batch)
    loss = loss_fun(preds, y_batch)


from sklearn.metrics import mean_absolute_error, accuracy_score
print(f'MAE: {mean_absolute_error(preds, y_test)}')


Epoch: 0, Loss: 26979197.40603125
Epoch: 10, Loss: 307.99179821777346
Epoch: 20, Loss: 161.80907023620605
Epoch: 30, Loss: 71.6918281211853
Epoch: 40, Loss: 171.68207153701783
Epoch: 50, Loss: 15.70089572763443
Epoch: 60, Loss: 35.4481316614151
Epoch: 70, Loss: 149.29318988275529
Epoch: 80, Loss: 145.8212443408966
Epoch: 90, Loss: 276.83085823822023
Epoch: 100, Loss: 52.39116136729717
Epoch: 110, Loss: 4.9637132139205935
Epoch: 120, Loss: 46.758320457458495
Epoch: 130, Loss: 838.1369025220871
Epoch: 140, Loss: 210.1647206811905
Epoch: 150, Loss: 328.83247338724135
Epoch: 160, Loss: 82.4009738368988
Epoch: 170, Loss: 503.45634974098203
Epoch: 180, Loss: 5.2192321263551715
Epoch: 190, Loss: 4.3521024502515795
MAE: 18.534379299163817


In [ ]:
preds.shape

torch.Size([2000, 1])

In [ ]:
y_test.shape

torch.Size([2000, 1])

In [ ]:
preds[0]

tensor([10036.5059])

In [ ]:
y_test[0]

tensor([10001.])

In [ ]:
model(torch.tensor([[1, 2]], dtype=torch.float))

tensor([[3.0610]], grad_fn=<AddmmBackward0>)

In [ ]:
model(torch.tensor([[1, 2]], dtype=torch.float))

tensor([[3.0610]], grad_fn=<AddmmBackward0>)

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# create data
N = 20000

# Generate data
data = torch.randint(low=-10000, high=10001, size=(N, 2)).float()
data

tensor([[-6754., -3687.],
        [-3759.,  8944.],
        [ 3278.,  4545.],
        ...,
        [ 8419.,  8108.],
        [-4922., -8806.],
        [ 5401., -6364.]])

In [ ]:
# labels
labels = torch.sum(data, axis=1, keepdim=True)
labels

tensor([[-10441.],
        [  5185.],
        [  7823.],
        ...,
        [ 16527.],
        [-13728.],
        [  -963.]])

In [ ]:
for i in range(10):
  print(data[i], labels[i])

tensor([-6754., -3687.]) tensor([-10441.])
tensor([-3759.,  8944.]) tensor([5185.])
tensor([3278., 4545.]) tensor([7823.])
tensor([8208., 6713.]) tensor([14921.])
tensor([1401., -508.]) tensor([893.])
tensor([3413., 2987.]) tensor([6400.])
tensor([ 5800., -9543.]) tensor([-3743.])
tensor([ 5635., -9580.]) tensor([-3945.])
tensor([-830., 7472.]) tensor([6642.])
tensor([-6932., -3503.]) tensor([-10435.])


In [ ]:
# a function that builds the model
def createADDmodel():

  ADDclassify = nn.Sequential(
      nn.Linear(2, 30), # input layer
      nn.ReLU(),        # activation unit
      nn.Linear(30, 1), # hidden layer
      nn.ReLU(),        # activation unit
      nn.Linear(1, 1)
  )

  # loss function
  lossfun = nn.MSELoss()

  # optimizer
  optimizer = torch.optim.Adam(ADDclassify.parameters(), lr=0.01)

  # model output
  return ADDclassify, lossfun, optimizer

In [ ]:
# a function that trains the model

# a fixed number of epochs
num_epochs = 10

def trainTheModel(ADDmodel, lossfun, optimizer):
  # initialize losses
  losses = torch.zeros(num_epochs)

  # loop over epochs
  for epoch in range(num_epochs):
    numloss = []
    # loop through 'minibatches' of N = 1
    for num, ans in zip(data, labels):
      # forward pass
      yhat = ADDmodel(num)

      # compute loss
      loss = lossfun(yhat, ans)
      numloss.append(loss.item())

      # backprop
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    losses[epoch] = np.mean(numloss)

    # final forward pass
    with torch.no_grad():
      predictions = ADDmodel(data)

    # compute the predictions and report accuracy
    trainacc = 100*torch.mean( (np.abs(predictions - labels) < 0.1).float())

    ## now test with new data
    # create brand new test data
    TESTdata = torch.randint(low=-10000, high=10001, size=(N, 2)).float()
    TESTlabels = torch.sum(TESTdata, axis=1, keepdim=True)

    # evaluate the model and compute accuracy
    with torch.no_grad():
      predictions = ADDmodel(TESTdata)
    testacc = 100 * torch.mean( (np.abs(predictions - TESTlabels) < 0.1).float() )

    # EOF
    return losses, trainacc, testacc, ADDmodel

In [ ]:
# test the model once to make sure it runs
AddingMachine, lossfun, optimizer = createADDmodel()
losses, trainacc, testacc, ADDmodel = trainTheModel(AddingMachine, lossfun, optimizer)

/tmp/ipython-input-36-1110496874.py:34: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  trainacc = 100*torch.mean( (np.abs(predictions - labels) < 0.1).float())
/tmp/ipython-input-36-1110496874.py:44: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  testacc = 100 * torch.mean( (np.abs(predictions - TESTlabels) < 0.1).float() )


In [ ]:
trainacc

tensor(0.0800)

In [ ]:
testacc

tensor(0.1000)

In [ ]:
# run the model 10 times to check reproducibility

for i in range(10):
  AddingMachine, lossfun, optimizer = createADDmodel()
  losses, trainacc, testacc, ADDmodel = trainTheModel(AddingMachine, lossfun, optimizer)
  print('Model instance %s, final TRAIN/TEST accuracies: %g%%, %g%%'%(i+1, trainacc, testacc))

/tmp/ipython-input-36-1110496874.py:34: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  trainacc = 100*torch.mean( (np.abs(predictions - labels) < 0.1).float())
/tmp/ipython-input-36-1110496874.py:44: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  testacc = 100 * torch.mean( (np.abs(predictions - TESTlabels) < 0.1).float() )


Model instance 1, final TRAIN/TEST accuracies: 0%, 0%
Model instance 2, final TRAIN/TEST accuracies: 0.035%, 0.025%
Model instance 3, final TRAIN/TEST accuracies: 0.04%, 0.02%
Model instance 4, final TRAIN/TEST accuracies: 0%, 0.015%
Model instance 5, final TRAIN/TEST accuracies: 0.02%, 0.005%
Model instance 6, final TRAIN/TEST accuracies: 0.025%, 0.02%
Model instance 7, final TRAIN/TEST accuracies: 0%, 0%
Model instance 8, final TRAIN/TEST accuracies: 0%, 0%
Model instance 9, final TRAIN/TEST accuracies: 0.02%, 0.025%
Model instance 10, final TRAIN/TEST accuracies: 0.03%, 0.02%
